# LedBade

## Setup the ESP

First, I downloaded the firmware to my Downloads-folder from [micropython.org](https://micropython.org/download/#esp8266).

(In the following if there is a '!' at the beginning of the line you don't have to enter it. It's there for technical reasons.)

In [1]:
!ls ~/Downloads/esp8266*bin

/Users/bakera/Downloads/esp8266-20180511-v1.9.4.bin


We can look for usb devices connected to our linux or macos machine. We need this information later for the command-line tools.

In [6]:
!ls /dev/*usb*

/dev/cu.wchusbserial1420  /dev/tty.wchusbserial1420


The following steps are taken from the [MicroPython documentation](http://docs.micropython.org/en/latest/esp8266/esp8266/tutorial/intro.html#deploying-the-firmware). So feel free to look into it if something goes wrong.

Installing the firmware takes two steps. First the flash will be erased.

In [4]:
!esptool.py --port /dev/tty.wchusbserial1420 erase_flash

esptool.py v2.5.0
Serial port /dev/tty.wchusbserial1420
Connecting....
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
MAC: bc:dd:c2:08:ad:7a
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 2.6s
Hard resetting via RTS pin...


Next, the new firmware will be installed.

In [7]:
!esptool.py --port /dev/tty.wchusbserial1420 --baud 115200 write_flash --flash_size=detect -fm dio 0 ~/Downloads/esp8266*

esptool.py v2.5.0
Serial port /dev/tty.wchusbserial1420
Connecting....
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
MAC: bc:dd:c2:08:ad:7a
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 604872 bytes to 394893...
Wrote 604872 bytes (394893 compressed) at 0x00000000 in 35.3 seconds (effective 137.2 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


## Install development tools

All tools are mentioned in ``requirements.txt``. First lets look into it.

In [7]:
cat requirements.txt

esptool
adafruit-ampy


Next we use pip to install them.

In [ ]:
!pip3 install -r requirements.txt

Let's have a look what we have installed.

First we have `ampy` to transfer files from and to the board.

In [2]:
!ampy

Usage: ampy [OPTIONS] COMMAND [ARGS]...

  ampy - Adafruit MicroPython Tool

  Ampy is a tool to control MicroPython boards over a serial connection.
  Using ampy you can manipulate files on the board's internal filesystem and
  even run scripts.

Options:
  -p, --port PORT    Name of serial port for connected board.  Can optionally
                     specify with AMPY_PORT environment variable.  [required]
  -b, --baud BAUD    Baud rate for the serial connection (default 115200).
                     Can optionally specify with AMPY_BAUD environment
                     variable.
  -d, --delay DELAY  Delay in seconds before entering RAW MODE (default 0).
                     Can optionally specify with AMPY_DELAY environment
                     variable.
  --version          Show the version and exit.
  --help             Show this message and exit.

Commands:
  get    Retrieve a file from the board.
  ls     List contents of a directory on the board.
  mkdir  Create a directory on

Then there is `esptool` for handling the firmware of the board.

In [3]:
!esptool.py

esptool.py v2.5.0
usage: esptool [-h] [--chip {auto,esp8266,esp32}] [--port PORT] [--baud BAUD]
               [--before {default_reset,no_reset,no_reset_no_sync}]
               [--after {hard_reset,soft_reset,no_reset}] [--no-stub]
               [--trace] [--override-vddsdio [{1.8V,1.9V,OFF}]]
               {load_ram,dump_mem,read_mem,write_mem,write_flash,run,image_info,make_image,elf2image,read_mac,chip_id,flash_id,read_flash_status,write_flash_status,read_flash,verify_flash,erase_flash,erase_region,version}
               ...

esptool.py v2.5.0 - ESP8266 ROM Bootloader Utility

positional arguments:
  {load_ram,dump_mem,read_mem,write_mem,write_flash,run,image_info,make_image,elf2image,read_mac,chip_id,flash_id,read_flash_status,write_flash_status,read_flash,verify_flash,erase_flash,erase_region,version}
                        Run esptool {command} -h for additional help
    load_ram            Download an image to RAM and execute
    dump_mem            Dump arbitrary memory t

## Fix for ampy

In order to run ampy on the *Wemos D1 mini* you must specify the
parameter ``--delay 0.5`` or ``-d 0.5`` on the commandline.
[Source](https://github.com/adafruit/ampy/issues/19).

    $ ampy -d 0.5 -p ... ls

## List files of the board

Now let's list the files on the device with the `ls` command.

In [8]:
!ampy -p /dev/tty.wchusbserial1420 -d 0.5 ls

boot.py


## (optional) Install development environment

For testing and runnning in a local micropython environment install 
the upython packages with

	$ micropython -m upip install -r urequirements.txt

## Connect to the device

You can connect to the device with ``screen`` on Linux/MacOS:

    $ screen /dev/tty... 115200

Screen can be exited with ``CTRL-A K``.

On Windows you can use [Putty](https://putty.org).

Try out some Python commands.

## Deploy files

We will now upload all files to the board (this may take some time).

In [9]:
%%bash

for f in src/*py; do
  echo uploading $f
  ampy -p /dev/tty.wchusbserial1420 -d 0.5 put $f
done

uploading src/boot.py
uploading src/ledmatrix.py
uploading src/main.py
uploading src/text.py
uploading src/tm1640.py
uploading src/tm1640_test.py
